In [69]:
import ollama
from ollama import chat

from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.chunking import HybridChunker

from pydantic.v1 import BaseModel

from transformers import AutoTokenizer
from sentence_transformers import CrossEncoder

from BM25 import load_bm25, create_bm25, bm25_search

import os
import json

In [70]:
# document local path or URL
doc = "./test_doc/test_textMore.pdf" # simple text with a image and table
doc1 = "./test_doc/吉美利101利率變動型美元終身壽險(定期給付型)DM.pdf" # complex text with simple aesthetic table
doc2 = "./test_doc/美利雙寶利率變動型美元終身保險 (定期給付型)DM.pdf" # complex text with complex aesthetic table
doc3 = "./test_doc/國泰金控出勤管理須知(修正後).pdf" # simple text with a simple table
doc4 = "./test_doc/國泰金控員工國內出差要點.pdf" # simple text with a simple table
doc5 = "./test_doc/國泰金控員工國外出差要點.pdf" # simple text with complex table
doc6 = "./test_doc/國泰金融控股股份有限公司資訊安全管理辦法.pdf" # simple text

# ollama embedding model
embed_model = "bge-m3:latest"

# reranker 
#rerank_model = CrossEncoder('BAAI/bge-reranker-large', max_length=512)
rerank_model = CrossEncoder('BAAI/bge-reranker-large')

# ollama llm
#llm = "deepseek-r1:7b"
#llm = "deepseek-r1:14b"
#llm = "deepseek-r1:32b"
llm = "deepseek-r1:14b-max-context"

def get_embeddings(texts, model=embed_model):
    embed_response = ollama.embeddings(model=model, prompt=texts)
    embedded_vector = embed_response["embedding"]
    
    return embedded_vector

# PyPdfium without EasyOCR
# --------------------
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = False
#pipeline_options.table_structure_options.do_cell_matching = True

#pipeline_options.table_structure_options.mode = 'accurate'

pipeline_options.images_scale = 2
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True

pypdfium_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
        pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
        )
    }
)

# print json
def show_json(data):
    if isinstance(data, str):
        obj = json.loads(data)
        print(json.dumps(obj, indent=4, ensure_ascii=False))
    elif isinstance(data, dict) or isinstance(data, list):
        print(json.dumps(data, indent=4, ensure_ascii=False))
    elif issubclass(type(data), BaseModel):
        print(json.dumps(data.dict(), indent=4, ensure_ascii=False))

# Docling reader, docling format parser output

In [73]:
doc_source = [doc, doc1, doc2, doc3, doc4, doc5, doc6]

conv_results = pypdfium_converter.convert_all(
    doc_source,
    raises_on_error=True,  # to let conversion run through all and examine results at the end
)

# Do hybrid chunking to merge similar chunk
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

hybrid_chunker = HybridChunker(
    tokenizer=tokenizer,
    #max_tokens=1024,
    merge_peers=True  # optional, defaults to True
)

all_chunks = []
for conv_res in conv_results:
    docling_docs = conv_res.document
    chunk_iter = hybrid_chunker.chunk(dl_doc=docling_docs)
    chunks = list(chunk_iter)
    all_chunks += chunks

In [74]:
for i, chunk in enumerate(all_chunks[:]):
    print(f"=== {i} ===")
    txt_tokens = len(tokenizer.tokenize(chunk.text))
    print(f"chunk.text ({txt_tokens} tokens):\n{repr(chunk.text)}")

    ser_txt = hybrid_chunker.serialize(chunk=chunk)
    ser_tokens = len(tokenizer.tokenize(ser_txt))
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")

    print()

=== 0 ===
chunk.text (285 tokens):
'（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 . 12 . 31 依 113.09.23 金管保壽字第 1130427324 號4 號函修正'
chunker.serialize(chunk) (311 tokens):
'國泰人壽祿壽祿美富利富利率變動型美元終身壽險（定期給付型）\n（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 

In [38]:
node_text, node_metadatas = [], []
for chunk in all_chunks:
    node_text.append(hybrid_chunker.serialize(chunk=chunk))
    node_metadatas.append(chunk.meta.export_json_dict())

In [39]:
embedded_text = []
for i in node_text:
    embedded_text.append(get_embeddings(i))

# Test on BM25 retrieval

In [40]:
#query = "去新加坡出差可以申請多少費用？"
#query = "科主管國內出差可以申請多少餐雜宿費？"
query = "系統開發之安全管理，應包含哪些項目？"

output_dir = 'test_BM25index'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# as json
bm25_mixed_json = create_bm25(node_text, 'mixed')
bm25_mixed_json.save(os.path.join(output_dir, 'bm25_mixed.json'))

loaded_bm25_mixed_json = load_bm25(os.path.join(output_dir, 'bm25_mixed.json'), node_text)

print("load from json and mixure search:", query)
print()
results_json = loaded_bm25_mixed_json.search(query, top_k=5)
for doc_id, score in results_json:
    print(f"Doc ID: {doc_id}, Score: {score:.4f}, Text: {node_text[doc_id]}\n")

load from json and mixure search: 系統開發之安全管理，應包含哪些項目？

Doc ID: 110, Score: 19.0649, Text: 資訊財訊財產安全管理作理作業
第四條 本公司資訊財訊財產安全管理作理作業之權之權責職掌應掌應適應適當劃分，並視需要建立 代理人制度。
本公司資司資訊財訊財產相產相關業務於委於委託「受「受委託者 」 辦理前理前，應與其簽其簽署簽署合署合 約並納並納入本辦法中「受「受委託者」應遵循之相之相關事項。
第五條 本公司員司員工之工之資訊安全教育訓練，應依下列規定辦理：
-、 各級主管須負責督導所屬員工之資訊作業安全，防範不法及不當 行為。
二、所、所有員有員工須依須依公司安排之課程參與資訊安全教育訓練，以瞭解維 護資訊安全為每位員工之義務，且於執行各項作業時，應確保所 執行作業之資訊安全，對執行業務過程中所得知之資訊，應嚴格 保密，其保密義務不因調離職而免而免除。
三、資訊安全部得視實際需要辦理或責成相關單位執行資訊安全教 育訓練及宣導，以建立員工資訊安全認知，並提升公司整體資訊 安全水準。
第六條 資訊財訊財產管理，應依下列規定辦理：
-、 本公司資司資訊財訊財產之使用 ，應遵應遵循本公司及司及受本公司委託管理維 運公司之相之相關規範。
二、 本公司嚴司嚴禁使禁使用來源不明之儲存媒體及非法軟體程式或未經公 司許司許可之軟體，所，所有員工及工及受及受委託者，均有責任及義務保護其所 取得或使用之本公司資訊財訊財產，並防止未經授權之存之存取、擅改、 破壞或不當揭露，以確保資訊財訊財產之機密性、完整性及可用性。
三、 本公司資 司資 訊 財訊 財產 應由應由資訊處或受委託者安裝管控軟體及體及防毒
第 2 頁2 頁/共 8 頁8 頁
軟體並開啟即時掃瞄功能，惟若符若符合《辦公室資訊作業管理要 點》規範之例之例外狀況，得依得依該依該要點辦理。
四、 本公司資訊財訊財產應由應由資訊處或受委託者啟者啟動存動存取軌取軌跡，惟若有若有 將資訊複訊複製或製或移或移動至外至外部儲存裝置之需求者求者，應遵循《辦公室資 訊作業管理要點》進行申行申請並取並取得核准，資訊處或受委託者並須並須 將複將複製或製或移或移動之資訊留訊留存紀錄。
五、 本公司硬司硬體資體資訊財訊財產，如經評估不堪使用或用或不再不再使再使用使用時用時，應依應

# Qdrant


In [41]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import PointStruct

class qdrant_DBConnector:
    def __init__(self, collection_name):#, embedding_fn):
        self.qdrant_client = QdrantClient("http://localhost:6333")

        self.collection_name = collection_name

        # create collection
        self.collection = self.qdrant_client.recreate_collection(
            collection_name = collection_name,
            vectors_config = models.VectorParams(
                distance = models.Distance.COSINE,
                size=len(get_embeddings("你好"))),
            optimizers_config = models.OptimizersConfigDiff(memmap_threshold=20000),
            hnsw_config = models.HnswConfigDiff(on_disk=True, m=16, ef_construct=100)
        )
        '''
        # embed models
        self.embedding_fn = embedding_fn
        qdrant_client.set_model(self.embedding_fn)
        '''

    ''' WARNING: NO CHECK ON EQUAL LENGTH YET'''
    def upsert_vector(self, vectors, data):
        # insert 'points' to qdrant by vector, 
        # payload with original text and metadata
        for i, vector in enumerate(vectors):
            self.qdrant_client.upsert(
                collection_name=self.collection_name,
                points=[PointStruct(id=i,
                                    vector=vectors[i],
                                    payload={"text": data.text[i],
                                             "metadata": data.metadata[i]})]
                                    
            )

        print("upsert finish")

    def retrieved_all(self):
        count_points = self.qdrant_client.count(
            collection_name=self.collection_name,
            exact=True,
        )
        result = self.qdrant_client.retrieve(
            collection_name=self.collection_name,
            ids=list(range(0, count_points.count)),
        )
        return result

    def vector_search(self, vector, top_k):
        # vector search qdrant DB
        result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            limit=top_k,
            append_payload=True,
        )
        return result
    
    def vector_search_json(self, vector, top_k):
        # vector search qdrant DB with json format output
        result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            limit=top_k,
            append_payload=True,
        )
        vector_result_json = {
            f"chunk_{item.id}": {
                "text": item.payload['text'],
                "rank": index,
                "score": item.score
            }
            for index, item in enumerate(result)
        }

        return vector_result_json

In [42]:
class DataObject:
    def __init__(self, text, metadata):
        self.text = text
        self.metadata = metadata

data = DataObject(node_text, node_metadatas)
data.text[0]

'國泰人壽祿壽祿美富利富利率變動型美元終身壽險（定期給付型）\n（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 . 12 . 31 依 113.09.23 金管保壽字第 1130427324 號4 號函修正'

In [43]:
# create a qdrant vectorDB object
vector_db = qdrant_DBConnector("qdrant_test")
# add data to db
vector_db.upsert_vector(embedded_text, data)

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_33263/4280916858.py:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  self.collection = self.qdrant_client.recreate_collection(


upsert finish


In [44]:
#query = "國內出差費用標準"
#query = "國外出差，附表五 "
query = "系統開發之安全管理，應包含哪些項目？"

embedded_query = get_embeddings(query)
results = vector_db.vector_search(embedded_query, 5)
print(f"尋找 「{query}」:")
#print(results[0])

for i, point in enumerate(results):
    print(f"\n🔹 Score: {point.score:.2f} - point ID: {point.id}:")
    print(point.payload['text'])
    print()


尋找 「系統開發之安全管理，應包含哪些項目？」:

🔹 Score: 0.51 - point ID: 109:
用詞定義
第三條 本辦法相法相關用詞定義如下：
第 1 頁1 頁/共 8 頁8 頁
資訊安全：目的在確保資訊的機密性、完整性、可用性、 私密性及合法性 ， 使資訊能安 全地 、正確地、適切地 及 可靠地被運用在達成本公司經 營目 標之規劃、執行、 管 理及相關作為上。


🔹 Score: 0.51 - point ID: 107:
主旨
第-條 為強為強化國化國泰金融控股股份有限公司(以下簡稱本公司)之資訊安全管 理，建立安全及可信賴之資訊作業環境，確保資保資訊財訊財產之機密性、完 整性及可用性，並提升同仁對資訊安全之認知， 以保以保障員障員工、客戶 、 合作夥伴與本公司之權益，依據｢國泰金融控股股份有限公司資訊安 全政策｣訂｣訂定資訊安全管理辦法(以下簡稱本辦法)。


🔹 Score: 0.49 - point ID: 106:
國泰金融控股股份有限公司資訊安全管全管理辦法
110年3月3月22日2日訂定 權責單位：資訊安全部


🔹 Score: 0.46 - point ID: 108:
適用對象及範及範圍
第二條 本辦法適法適用對象為本公司所有員工(含(含各級主管、實、實習生及工讀生，以 下同)及)及其他得接觸本公司資訊及訊及資訊財訊財產之合之合作夥伴、委託廠商(含(含 顧問) (以下合稱「受「受委託者」)等。
本辦法適用範用範圍如下，以下合稱資訊財訊財產(Information Assets)：
-、 本公司所有或有或持有之資之資料與文件。
二、 本公司使用之主機、伺服器、個人電腦 (含(含可攜式電腦，以 下同)、行動裝置、終端設備、通訊線路 、儲存設備(含(含可 讀寫CD裝置、外接硬碟 、 USB儲B儲存媒體、記憶卡等，以 下同)，以及與前述相關或是與本公司資司資訊系統相連之硬 體資訊財訊財產。
三、 本公司使用於存於存放資訊財訊財產之實體環境。
四、 本公司使用之資之資訊系統，以及與前述資述資訊系統相關或相 連之軟體資訊財訊財產。
五、 其他本公司所有或有或有權使權使用之資之資訊財訊財產。


🔹 Score: 0.46 - point ID: 73:
第十條十條（控管機制）
各單位主管應督導所屬確實遵守本須知，並掌握員工出、缺勤異常動

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_33263/4280916858.py:55: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = self.qdrant_client.search(


# Try chatting!

In [45]:
"""stream = chat(
    model='deepseek-r1:7b',
    messages=[{'role': 'user', 'content': '妳好'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)"""

"stream = chat(\n    model='deepseek-r1:7b',\n    messages=[{'role': 'user', 'content': '妳好'}],\n    stream=True,\n)\n\nfor chunk in stream:\n  print(chunk['message']['content'], end='', flush=True)"

In [46]:
def bm25_retrieval(query, top_k=3):
    result = vector_db.retrieved_all()
    retrieved_text = []
    for i, points in enumerate(result):
        retrieved_text.append(result[i].payload['text'])

    bm25_result = bm25_search(corpus=retrieved_text, query=query, top_k=top_k)
    bm25_result_json = {
        f"chunk_{item[0]}": {
            "text": item[2],
            "rank": index,
            "score": item[1]
        }
        for index, item in enumerate(bm25_result)
    }

    return bm25_result_json

def rrf(ranks, k=1):
    ret = {}
    # recursive through all retrieved method
    for rank in ranks:
        for id, val in rank.items():
            if id not in ret:
                ret[id] = {"score": 0, "text": val["text"]}
            # calculate rrf score
            ret[id]["score"] += 1.0/(k+val["rank"])
    # sort and return according to rrf score
    return dict(sorted(ret.items(), key=lambda item: item[1]["score"], reverse=True))

def get_completion(prompt, model=llm):
    messages = [{"role": "user", "content": prompt}]
    response = chat(
        model=model,
        messages=messages
        #stream=True,
        #format="json",
        #options={"temperature":0}
    )
    return response.message.content


In [59]:
query = "去日本出差可以申請多少費用？"

print("search for:", query)
bm25_retrieve_result = bm25_retrieval(query)
show_json(bm25_retrieve_result)

search for: 去日本出差可以申請多少費用？
{
    "chunk_82": {
        "text": "第七條費(費條(費用核銷)\n員工公差完畢應即返回任所銷差，並於七日內於國內出差費用核銷系統提出費用申請，並檢 附相關收據，報請費用核銷。",
        "rank": 0,
        "score": 9.422666275980358
    },
    "chunk_104": {
        "text": "附表五、海外境內差旅 - 宿費及費及雜費標費標準\n單位：美元/日/日\n1, 出差地.費用類別 = 新加坡 (Singapore). 1, 高階主管及  總經理級以上主管.宿費 = 400. 1, 高階主管及  總經理級以上主管.雜費 = 檢據實支. 1, 協理級(含)以下人員  宿費 雜費.宿費 = 320. 1, 協理級(含)以下人員  宿費 雜費.雜費 = 20. 2, 出差地.費用類別 = 香港(Hong Kong). 2, 高階主管及  總經理級以上主管.宿費 = 350. 2, 高階主管及  總經理級以上主管.雜費 = 檢據實支. 2, 協理級(含)以下人員  宿費 雜費.宿費 = 280. 2, 協理級(含)以下人員  宿費 雜費.雜費 = 18. 3, 出差地.費用類別 = 菲律賓 (Philippines). 3, 高階主管及  總經理級以上主管.宿費 = 260. 3, 高階主管及  總經理級以上主管.雜費 = 檢據實支. 3, 協理級(含)以下人員  宿費 雜費.宿費 = 210. 3, 協理級(含)以下人員  宿費 雜費.雜費 = 13. 4, 出差地.費用類別 = 泰國 (Thailand). 4, 高階主管及  總經理級以上主管.宿費 = 245. 4, 高階主管及  總經理級以上主管.雜費 = 檢據實支. 4, 協理級(含)以下人員  宿費 雜費.宿費 = 200. 4, 協理級(含)以下人員  宿費 雜費.雜費 = 13. 5, 出差地.費用類別 = 馬來西亞 (Malaysia). 5, 高階主管及  總經理級以上主管.宿費 = 260. 5, 高階主管及  總經理級以上主管.雜費 = 檢據實支. 5, 協理級(含)以下人員  宿費 雜費.宿費 = 190. 5, 協理級(

In [60]:
embedded_query = get_embeddings(query)
vector_result_json = vector_db.vector_search_json(embedded_query, 3)
print("search for:", query)

show_json(vector_result_json)

search for: 去日本出差可以申請多少費用？
{
    "chunk_88": {
        "text": "第三條(餐(餐雜宿費標準)\n員工跨境出差之宿費及餐雜費悉費悉依附表-標準支給之。\n第四條(飛(飛機艙別及其他交通工具搭乘標準及交通費)\n奉派出國人員往返機票費及其他交通工具費用准予按實報支，其標準如附表二。\n在國外必須之交通，應依經奉准旅程表之路程為之，其交通費須檢附收據按實報支，自辦公 (住(住宿)處)處往返國際機場且來回車資，副總經理(含(含)以上實報實銷，協理級(含(含)以下同仁當仁當次出 差往返國內外機場之費用總計上限新限新台幣六千六千元得檢據核銷。",
        "rank": 0,
        "score": 0.6089779
    },
    "chunk_84": {
        "text": "第八條(修(修訂及施行)\n附件：國內出差雜、宿、交通費標準\n總經理以上, 全日雜費 = 檢據實支. 總經理以上, 宿費 = 檢據實支. 總經理以上, 交通費 = 檢據實支. 高階主管, 全日雜費 = 檢據實支. 高階主管, 宿費 = 檢據實支. 高階主管, 交通費 = 檢據實支. 部室主管及相當職級, 全日雜費 = 每日 300 元. 部室主管及相當職級, 宿費 = 元核支交通費。. 部室主管及相當職級, 交通費 = 1. 以飛機經濟艙、高鐵  標準車廂或自強號之. 科主管及相當職級, 全日雜費 = 每日 300 元. 科主管及相當職級, 宿費 = 元核支交通費。. 科主管及相當職級, 交通費 = 價格為上限。  2. 如自行開車者，依自  強號每公計. 其他人員, 全日雜費 = 每日 300 元. 其他人員, 宿費 = 元核支交通費。. 其他人員, 交通費 = 元核支交通費。  3. 搭乘計程車往返機場  或車站者，補貼計程  車 費 用 單 程 上 限  350 元，檢據實支。",
        "rank": 1,
        "score": 0.60871416
    },
    "chunk_85": {
        "text": "第八條(修(修訂及施行)\n備註：\n-、本公司員工於國內出差時應檢具實支，金額上限如前表；無法檢具實支者，則以 金額上限支給。\n二、本

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_33263/4280916858.py:65: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = self.qdrant_client.search(


In [61]:
hybrid_result = rrf([vector_result_json, bm25_retrieve_result])

print(json.dumps(hybrid_result, indent=4, ensure_ascii=False))

{
    "chunk_88": {
        "score": 1.0,
        "text": "第三條(餐(餐雜宿費標準)\n員工跨境出差之宿費及餐雜費悉費悉依附表-標準支給之。\n第四條(飛(飛機艙別及其他交通工具搭乘標準及交通費)\n奉派出國人員往返機票費及其他交通工具費用准予按實報支，其標準如附表二。\n在國外必須之交通，應依經奉准旅程表之路程為之，其交通費須檢附收據按實報支，自辦公 (住(住宿)處)處往返國際機場且來回車資，副總經理(含(含)以上實報實銷，協理級(含(含)以下同仁當仁當次出 差往返國內外機場之費用總計上限新限新台幣六千六千元得檢據核銷。"
    },
    "chunk_82": {
        "score": 1.0,
        "text": "第七條費(費條(費用核銷)\n員工公差完畢應即返回任所銷差，並於七日內於國內出差費用核銷系統提出費用申請，並檢 附相關收據，報請費用核銷。"
    },
    "chunk_84": {
        "score": 0.5,
        "text": "第八條(修(修訂及施行)\n附件：國內出差雜、宿、交通費標準\n總經理以上, 全日雜費 = 檢據實支. 總經理以上, 宿費 = 檢據實支. 總經理以上, 交通費 = 檢據實支. 高階主管, 全日雜費 = 檢據實支. 高階主管, 宿費 = 檢據實支. 高階主管, 交通費 = 檢據實支. 部室主管及相當職級, 全日雜費 = 每日 300 元. 部室主管及相當職級, 宿費 = 元核支交通費。. 部室主管及相當職級, 交通費 = 1. 以飛機經濟艙、高鐵  標準車廂或自強號之. 科主管及相當職級, 全日雜費 = 每日 300 元. 科主管及相當職級, 宿費 = 元核支交通費。. 科主管及相當職級, 交通費 = 價格為上限。  2. 如自行開車者，依自  強號每公計. 其他人員, 全日雜費 = 每日 300 元. 其他人員, 宿費 = 元核支交通費。. 其他人員, 交通費 = 元核支交通費。  3. 搭乘計程車往返機場  或車站者，補貼計程  車 費 用 單 程 上 限  350 元，檢據實支。"
    },
    "chunk_104": {
        "score": 0.5,
      

In [50]:
def hybrid_retriever(query, top_k=3):
    embedded_query = get_embeddings(query)
    result = rrf([vector_db.vector_search_json(embedded_query, top_k), bm25_retrieval(query, top_k)])
    return result

def reranker(query, retrieved_result, threshold=0):
    text_chunks = []
    for chunk_id, val in retrieved_result.items():
        text_chunks.append(val['text'])
    scores = rerank_model.predict([(query, doc) for doc in text_chunks])
    sorted_list = sorted(zip(scores, text_chunks), key=lambda x: x[0], reverse=True)

    return [chunk for chunk in sorted_list if chunk[0] > threshold]

def format_rag_output(reranked_list):
    formatted_docs = "\n\n".join([f"Document {i+1}:\n{text[1]}" for i, text in enumerate(reranked_list)])
    return formatted_docs

In [68]:
query = "去中國大陸出差可以申請多少費用？"
print(format_rag_output(reranker(query, hybrid_retriever(query, 20), 0.45)))

print()
for chunk in reranker(query, hybrid_retriever(query, 20)):
    print(chunk)
    print()

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_33263/4280916858.py:65: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = self.qdrant_client.search(


Document 1:
附表-：跨境差旅-宿費及餐雜費標準
單位：美元日/日元/日
亞洲地區, 總經理級以上主管   宿費 餐雜費.宿費 = . 亞洲地區, 總經理級以上主管   宿費 餐雜費.餐雜費 = . 亞洲地區, 高階主管.宿費 = . 亞洲地區, 高階主管.餐雜費 = . 亞洲地區, 協理級(含)   以下人員   宿費 餐雜費.宿費 = . 亞洲地區, 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = . 中國大陸, 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 中國大陸, 總經理級以上主管   宿費 餐雜費.餐雜費 = 110或檢據實支. 中國大陸, 高階主管.宿費 = 270. 中國大陸, 高階主管.餐雜費 = 110. 中國大陸, 協理級(含)   以下人員   宿費 餐雜費.宿費 = 215. 中國大陸, 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 95. 日本(Japan), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 日本(Japan), 總經理級以上主管   宿費 餐雜費.餐雜費 = 140或檢據實支. 日本(Japan), 高階主管.宿費 = 345. 日本(Japan), 高階主管.餐雜費 = 140. 日本(Japan), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 275. 日本(Japan), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 105. 南韓(Korea), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 南韓(Korea), 總經理級以上主管   宿費 餐雜費.餐雜費 = 130或檢據實支. 南韓(Korea), 高階主管.宿費 = 315. 南韓(Korea), 高階主管.餐雜費 = 130. 南韓(Korea), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 255. 南韓(Korea), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 110. 菲律賓(Philippines), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 菲律賓(Philippines), 總經理級以上主管   宿費 餐雜費.餐雜費 = 105或檢據實支. 菲律賓(Philippines), 高階主管.宿費 = 260. 菲律賓

In [66]:
# Job instruction
instruction = """
你是台灣國泰金控的聊天機器人秘書，專門為用戶解答公司內部的差旅費用、報銷標準等相關問題。
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
3. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。
"""

# User input
input_text = "去中國大陸出差可以申請多少費用？"
#input_text = "去日本出差可以申請多少費用？"
#input_text = "去新加坡出差可以申請多少費用？"
#input_text = "科主管國內出差可以申請多少餐雜宿費？"


# RAG retrieved documents
reranked_list = reranker(input_text, hybrid_retriever(input_text, 20), 0.45)
rag_docs = format_rag_output(reranked_list)

# Prompt template
prompt = f"""
# 任務
{instruction}

# 參考文件
{rag_docs}

# 用戶問題
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# llm calling
if len(rag_docs) != 0:
    response = get_completion(prompt)
else:
    response = "YAh, you retrieved NOTHING!"
print(response)

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_33263/4280916858.py:65: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = self.qdrant_client.search(


==== Prompt ====

# 任務

你是台灣國泰金控的聊天機器人秘書，專門為用戶解答公司內部的差旅費用、報銷標準等相關問題。
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
3. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。


# 參考文件
Document 1:
附表-：跨境差旅-宿費及餐雜費標準
單位：美元日/日元/日
亞洲地區, 總經理級以上主管   宿費 餐雜費.宿費 = . 亞洲地區, 總經理級以上主管   宿費 餐雜費.餐雜費 = . 亞洲地區, 高階主管.宿費 = . 亞洲地區, 高階主管.餐雜費 = . 亞洲地區, 協理級(含)   以下人員   宿費 餐雜費.宿費 = . 亞洲地區, 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = . 中國大陸, 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 中國大陸, 總經理級以上主管   宿費 餐雜費.餐雜費 = 110或檢據實支. 中國大陸, 高階主管.宿費 = 270. 中國大陸, 高階主管.餐雜費 = 110. 中國大陸, 協理級(含)   以下人員   宿費 餐雜費.宿費 = 215. 中國大陸, 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 95. 日本(Japan), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 日本(Japan), 總經理級以上主管   宿費 餐雜費.餐雜費 = 140或檢據實支. 日本(Japan), 高階主管.宿費 = 345. 日本(Japan), 高階主管.餐雜費 = 140. 日本(Japan), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 275. 日本(Japan), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 105. 南韓(Korea), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 南韓(Korea), 總經理級以上主管   宿費 餐雜費.餐雜費 = 130或檢據實支. 南韓(Korea), 高

In [53]:
test_text = '''附表-：跨境差旅-宿費及餐雜費標準
單位：美元日/日元/日
亞洲地區, 總經理級以上主管   宿費 餐雜費.宿費 = . 亞洲地區, 總經理級以上主管   宿費 餐雜費.餐雜費 = . 亞洲地區, 高階主管.宿費 = . 亞洲地區, 高階主管.餐雜費 = . 亞洲地區, 協理級(含)   以下人員   宿費 餐雜費.宿費 = . 亞洲地區, 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = . 中國大陸, 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 中國大陸, 總經理級以上主管   宿費 餐雜費.餐雜費 = 110或檢據實支. 中國大陸, 高階主管.宿費 = 270. 中國大陸, 高階主管.餐雜費 = 110. 中國大陸, 協理級(含)   以下人員   宿費 餐雜費.宿費 = 215. 中國大陸, 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 95. 日本(Japan), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 日本(Japan), 總經理級以上主管   宿費 餐雜費.餐雜費 = 140或檢據實支. 日本(Japan), 高階主管.宿費 = 345. 日本(Japan), 高階主管.餐雜費 = 140. 日本(Japan), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 275. 日本(Japan), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 105. 南韓(Korea), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 南韓(Korea), 總經理級以上主管   宿費 餐雜費.餐雜費 = 130或檢據實支. 南韓(Korea), 高階主管.宿費 = 315. 南韓(Korea), 高階主管.餐雜費 = 130. 南韓(Korea), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 255. 南韓(Korea), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 110. 菲律賓(Philippines), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 菲律賓(Philippines), 總經理級以上主管   宿費 餐雜費.餐雜費 = 105或檢據實支. 菲律賓(Philippines), 高階主管.宿費 = 260. 菲律賓(Philippines), 高階主管.餐雜費 = 105. 菲律賓(Philippines), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 210. 菲律賓(Philippines), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 85. 泰國(Thailand), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 泰國(Thailand), 總經理級以上主管   宿費 餐雜費.餐雜費 = 100或檢據實支. 泰國(Thailand), 高階主管.宿費 = 245. 泰國(Thailand), 高階主管.餐雜費 = 100. 泰國(Thailand), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 200. 泰國(Thailand), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 85. 馬來西亞(Malaysia), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 馬來西亞(Malaysia), 總經理級以上主管   宿費 餐雜費.餐雜費 = 100或檢據實支. 馬來西亞(Malaysia), 高階主管.宿費 = 260. 馬來西亞(Malaysia), 高階主管.餐雜費 = 100. 馬來西亞(Malaysia), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 190. 馬來西亞(Malaysia), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 95. 新加坡(Singapore), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 新加坡(Singapore), 總經理級以上主管   宿費 餐雜費.餐雜費 = 160或檢據實支. 新加坡(Singapore), 高階主管.宿費 = 400. 新加坡(Singapore), 高階主管.餐雜費 = 160. 新加坡(Singapore), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 320. 新加坡(Singapore), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 140. 印尼(Indonesia), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 印尼(Indonesia), 總經理級以上主管   宿費 餐雜費.餐雜費 = 100或檢據實支. 印尼(Indonesia), 高階主管.宿費 = 260. 印尼(Indonesia), 高階主管.餐雜費 = 100. 印尼(Indonesia), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 190. 印尼(Indonesia), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 95.
緬甸(Burma), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 緬甸(Burma), 總經理級以上主管   宿費 餐雜費.餐雜費 = 100或檢據實支. 緬甸(Burma), 高階主管.宿費 = 260. 緬甸(Burma), 高階主管.餐雜費 = 100. 緬甸(Burma), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 190. 緬甸(Burma), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 95.
'''

#緬甸(Burma), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 緬甸(Burma), 總經理級以上主管   宿費 餐雜費.餐雜費 = 100或檢據實支. 緬甸(Burma), 高階主管.宿費 = 260. 緬甸(Burma), 高階主管.餐雜費 = 100. 緬甸(Burma), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 190. 緬甸(Burma), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 95.
#印度(India), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 印度(India), 總經理級以上主管   宿費 餐雜費.餐雜費 = 120或檢據實支. 印度(India), 高階主管.宿費 = 300. 印度(India), 高階主管.餐雜費 = 120. 印度(India), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 240. 印度(India), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 105. 柬埔寨(Cambodia), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 柬埔寨(Cambodia), 總經理級以上主管   宿費 餐雜費.餐雜費 = 90或檢據實支. 柬埔寨(Cambodia), 高階主管.宿費 = 240. 柬埔寨(Cambodia), 高階主管.餐雜費 = 90. 柬埔寨(Cambodia), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 145. 柬埔寨(Cambodia), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 85.

txt_tokens = len(tokenizer.tokenize(test_text))
txt_tokens

1489

In [54]:
# Job instruction
instruction = """
你是台灣國泰金控的聊天機器人秘書，專門為用戶解答公司內部的差旅費用、報銷標準等相關問題。
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
3. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。
"""

# User input
input_text = "去日本出差可以申請多少費用？"

# RAG retrieved documents
reranked_list = reranker(input_text, hybrid_retriever(input_text, 20), 0.45)
rag_docs = format_rag_output(reranked_list)

# Prompt template
prompt = f"""

# 參考文件
{test_text}

# 用戶問題
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# llm calling
response = get_completion(prompt)
print(response)

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_33263/4280916858.py:65: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = self.qdrant_client.search(


==== Prompt ====


# 參考文件
附表-：跨境差旅-宿費及餐雜費標準
單位：美元日/日元/日
亞洲地區, 總經理級以上主管   宿費 餐雜費.宿費 = . 亞洲地區, 總經理級以上主管   宿費 餐雜費.餐雜費 = . 亞洲地區, 高階主管.宿費 = . 亞洲地區, 高階主管.餐雜費 = . 亞洲地區, 協理級(含)   以下人員   宿費 餐雜費.宿費 = . 亞洲地區, 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = . 中國大陸, 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 中國大陸, 總經理級以上主管   宿費 餐雜費.餐雜費 = 110或檢據實支. 中國大陸, 高階主管.宿費 = 270. 中國大陸, 高階主管.餐雜費 = 110. 中國大陸, 協理級(含)   以下人員   宿費 餐雜費.宿費 = 215. 中國大陸, 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 95. 日本(Japan), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 日本(Japan), 總經理級以上主管   宿費 餐雜費.餐雜費 = 140或檢據實支. 日本(Japan), 高階主管.宿費 = 345. 日本(Japan), 高階主管.餐雜費 = 140. 日本(Japan), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 275. 日本(Japan), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 105. 南韓(Korea), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 南韓(Korea), 總經理級以上主管   宿費 餐雜費.餐雜費 = 130或檢據實支. 南韓(Korea), 高階主管.宿費 = 315. 南韓(Korea), 高階主管.餐雜費 = 130. 南韓(Korea), 協理級(含)   以下人員   宿費 餐雜費.宿費 = 255. 南韓(Korea), 協理級(含)   以下人員   宿費 餐雜費.餐雜費 = 110. 菲律賓(Philippines), 總經理級以上主管   宿費 餐雜費.宿費 = 檢據實支. 菲律賓(Philippines), 總經理級以上主管   宿費 餐雜費.餐雜費 = 105或檢據實支. 菲律賓(Philippines), 高階主管